# Revive it
Checking on air quality in Barcelona so we make it a healthier place.

## Dataset
We're using the official data from the city of Barcelona: https://opendata-ajuntament.barcelona.cat/data/en/dataset/qualitat-aire-detall-bcn
As data up to April 2019 is in a different format, we only use data from April 2019 up to the (currently) most recent data: February 2024.

## Check the data and put it in one document

In [9]:
# Importing stuff

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

2024-04-04 13:53:10.724323: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 13:53:10.779788: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 13:53:10.943335: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 13:53:16.485860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [20]:
# Putting data in one excel and saving it for futher use
data = pd.DataFrame()

all_csv = [csv for csv in os.listdir("../raw_data") if csv.endswith(".csv")]

for csv in all_csv:
    print(f"starting {csv}")
    new_df = data
    df = pd.read_csv(f"../raw_data/{csv}")
    data = pd.concat([new_df, df])

data.to_csv("../data/data.csv", index=False)

starting 2022_08_Agost_qualitat_aire_BCN.csv
starting 2023_11_Novembre_qualitat_aire_BCN.csv
starting 2023_12_Desembre_qualitat_aire_BCN.csv
starting 2019_10_Octubre_qualitat_aire_BCN.csv
starting 2023_03_Marc_qualitat_aire_BCN.csv
starting 2019_08_Agost_qualitat_aire_BCN.csv
starting 2023_08_Agost_qualitat_aire_BCN.csv
starting 2023_02_Febrer_qualitat_aire_BCN.csv
starting 2019_06_Juny_qualitat_aire_BCN.csv
starting 2023_10_Octubre_qualitat_aire_BCN.csv
starting 2023_05_Maig_qualitat_aire_BCN.csv
starting 2021_07_Juliol_qualitat_aire_BCN.csv
starting 2021_05_Maig_qualitat_aire_BCN.csv
starting 2021_10_Octubre_qualitat_aire_BCN.csv
starting 2023_04_Abril_qualitat_aire_BCN.csv
starting 2022_11_Novembre_qualitat_aire_BCN.csv
starting 2021_02_Febrer_qualitat_aire_BCN.csv
starting 2023_07_Juliol_qualitat_aire_BCN.csv
starting 2020_02_Febrer_qualitat_aire_BCN.csv
starting 2019_12_Desembre_qualitat_aire_BCN.csv
starting 2021_01_Gener_qualitat_aire_BCN.csv
starting 2021_08_Agost_qualitat_aire

In [21]:
# Uploaded saved excel with all data
test = pd.read_csv(f"../data/data.csv")
test.head()

/tmp/ipykernel_14818/2390811363.py:1: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(f"../data/data.csv")


,CODI_PROVINCIA,PROVINCIA,CODI_MUNICIPI,MUNICIPI,ESTACIO,CODI_CONTAMINANT,ANY,MES,DIA,H01,...,V20,H21,V21,H22,V22,H23,V23,H24,V24,CODI_PROVINCIA;PROVINCIA;CODI_MUNICIPI;MUNICIPI;ESTACIO;CODI_CONTAMINANT;ANY;MES;DIA;H01;V01;H02;V02;H03;V03;H04;V04;H05;V05;H06;V06;H07;V07;H08;V08;H09;V09;H10;V10;H11;V11;H12;V12;H13;V13;H14;V14;H15;V15;H16;V16;H17;V17;H18;V18;H19;V19;H20;V20;H21;V21;H22;V22;H23;V23;H24;V24
0,8.0,Barcelona,19.0,Barcelona,4.0,7.0,2022.0,8.0,1.0,1.0,...,V,2.0,V,1.0,V,2.0,V,1.0,V,NaN
1,8.0,Barcelona,19.0,Barcelona,4.0,7.0,2022.0,8.0,2.0,1.0,...,V,2.0,V,1.0,V,1.0,V,2.0,V,NaN
2,8.0,Barcelona,19.0,Barcelona,4.0,7.0,2022.0,8.0,3.0,1.0,...,V,1.0,V,1.0,V,1.0,V,1.0,V,NaN
3,8.0,Barcelona,19.0,Barcelona,4.0,7.0,2022.0,8.0,4.0,1.0,...,V,NaN,N,1.0,V,1.0,V,NaN,N,NaN
4,8.0,Barcelona,19.0,Barcelona,4.0,7.0,2022.0,8.0,5.0,1.0,...,V,1.0,V,1.0,V,1.0,V,1.0,V,NaN
